Text preprocessing

In [1]:
import os, re
import pandas as pd
import ufal.udpipe
import conllu

import my_parsers
import classifier_preprocessing
from voters import decision

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [2]:
%%time
decision('Борис взял и вышел.')

CPU times: user 12.6 ms, sys: 0 ns, total: 12.6 ms
Wall time: 199 ms


'TRUE'

In [3]:
DATA = os.path.join(os.getcwd(), 'Gold_standard_for_validation.txt')

In [4]:
with open(DATA, mode='r', encoding='utf-16') as data:
    corpus = data.read()

In [5]:
with open(DATA, mode='r', encoding='utf-16') as data:
    df = pd.read_csv(data, sep='\t', header=None, names=['sentence', 'class'])

In [6]:
# df = df.drop(['nan'], axis=1).drop([181])

In [7]:
df

,sentence,class
0,"Борис сказал, что придёт.",coreference resolution
1,Борис взял и начал читать.,TRUE
2,Борис взял и он начал читать.,FALSE
3,Борис встал и вышел.,TRUE
4,Борис встал и он вышел.,TRUE
...,...,...
176,"Я скоро увижу Лену и я искренно надеюсь, что о...",TRUE
177,"Я только что видел Лену.Она сказала, что наш д...",TRUE
178,"Я только что видел Лену.Сказала, что наш дом п...",FALSE
179,"Я только что встретил Лену.Cказала, что их отд...",FALSE


In [8]:
df['sentence'] = df['sentence'].apply(lambda x: classifier_preprocessing.preprocess(x))

In [9]:
%%time
df['evaluation1'] = df['sentence'].apply(lambda x: decision(x))
df['evaluation2'] = df['sentence'].apply(lambda x: decision(x))
df['evaluation3'] = df['sentence'].apply(lambda x: decision(x))
df['evaluation4'] = df['sentence'].apply(lambda x: decision(x))
df['evaluation5'] = df['sentence'].apply(lambda x: decision(x))

CPU times: user 4.59 s, sys: 396 ms, total: 4.99 s
Wall time: 2min 50s


In [10]:
df

,sentence,class,evaluation1,evaluation2,evaluation3,evaluation4,evaluation5
0,"Борис сказал, что придёт.",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution
1,Борис взял и начал читать.,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
2,Борис взял и он начал читать.,FALSE,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution
3,Борис встал и вышел.,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
4,Борис встал и он вышел.,TRUE,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution
...,...,...,...,...,...,...,...
176,"Я скоро увижу Лену и я искренно надеюсь, что о...",TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
177,"Я только что видел Лену.Она сказала, что наш д...",TRUE,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution
178,"Я только что видел Лену.Сказала, что наш дом п...",FALSE,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution
179,"Я только что встретил Лену.Cказала, что их отд...",FALSE,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution


In [11]:
df[df['evaluation1'] == 'None']

,sentence,class,evaluation1,evaluation2,evaluation3,evaluation4,evaluation5


In [12]:
decision("Борис сказал, что ей холодно.")

'coreference resolution'

In [13]:
df['numclass'] = df['class'].apply(lambda x: classifier_preprocessing.class2id(x))
df['numeval1'] = df['evaluation1'].apply(lambda x: int(classifier_preprocessing.class2id(x)))
df['numeval2'] = df['evaluation2'].apply(lambda x: int(classifier_preprocessing.class2id(x)))
df['numeval3'] = df['evaluation3'].apply(lambda x: int(classifier_preprocessing.class2id(x)))
df['numeval4'] = df['evaluation4'].apply(lambda x: int(classifier_preprocessing.class2id(x)))
df['numeval5'] = df['evaluation5'].apply(lambda x: int(classifier_preprocessing.class2id(x)))

In [14]:
df[df['numclass'] == 2]

,sentence,class,evaluation1,evaluation2,evaluation3,evaluation4,evaluation5,numclass,numeval1,numeval2,numeval3,numeval4,numeval5
0,"Борис сказал, что придёт.",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
5,"Борис встретился с Леной.Говорит, что их отдел...",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
8,"Борис объявил, что готов.",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
11,"Борис ответил, что придет завтра.",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
78,"Лена разговаривает с Борисом.Говорит, что отде...",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
160,"Саша обедает с Сашей.Сказала, что отдел скоро ...",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2
172,"Я сказал ему, чтобы приходил поскорее.",coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,coreference resolution,2,2,2,2,2,2


In [15]:
hand_rule_target = df['numclass']
pred1 = df['numeval1']
pred2 = df['numeval2']
pred3 = df['numeval3']
pred4 = df['numeval4']
pred5 = df['numeval5']
predictions = [pred1, pred2, pred3, pred4, pred5]

In [16]:
def precision_mean(predictions: list):
    results = list()
    for pred in predictions:
        results.append(precision_score(hand_rule_target, pred, average='weighted'))
    return 'Precision mean: %.4f' % (sum(results) / len(results))

In [17]:
def recall_mean(predictions: list):
    results = list()
    for pred in predictions:
        results.append(recall_score(hand_rule_target, pred, average='weighted'))
    return 'Recall mean: %.4f' % (sum(results) / len(results))

In [35]:
def f1_mean(predictions: list):
    results = list()
    for pred in predictions:
        results.append(f1_score(hand_rule_target, pred, average='micro'))
    return 'F1 mean: %.4f' % (sum(results) / len(results))   

In [32]:
precision_mean(predictions)

'Precision mean: 0.7195'

In [33]:
recall_mean(predictions)

'Recall mean: 0.3481'

In [36]:
f1_mean(predictions)

'F1 mean: 0.3481'

In [22]:
print(classification_report(hand_rule_target, pred1))

              precision    recall  f1-score   support

           0       0.40      0.21      0.28        38
           1       0.86      0.36      0.51       136
           2       0.07      1.00      0.13         7

    accuracy                           0.35       181
   macro avg       0.44      0.52      0.30       181
weighted avg       0.73      0.35      0.44       181



In [23]:
%%time
decision('Борис сказал, что она богата.')

CPU times: user 6.07 ms, sys: 300 µs, total: 6.37 ms
Wall time: 185 ms


'coreference resolution'

In [24]:
%%time
decision('Борис сказал, что он придет.')

CPU times: user 6.54 ms, sys: 0 ns, total: 6.54 ms
Wall time: 171 ms


'coreference resolution'

In [25]:
%%time
decision('Борис взял и вышел.')

CPU times: user 5.41 ms, sys: 201 µs, total: 5.61 ms
Wall time: 160 ms


'TRUE'

In [26]:
%%time 
decision('Миша узнал, что нужно делать, когда он придёт домой.')

CPU times: user 3.25 ms, sys: 3.93 ms, total: 7.18 ms
Wall time: 205 ms


'FALSE'

In [27]:
%%time 
decision('Борис сказал, что богат.')

CPU times: user 7.64 ms, sys: 0 ns, total: 7.64 ms
Wall time: 167 ms


'coreference resolution'

In [28]:
%%time
decision("Борис сказал, он зайдет.")

CPU times: user 5.61 ms, sys: 230 µs, total: 5.84 ms
Wall time: 189 ms


'coreference resolution'

In [29]:
%%time 
decision("Он сел рядом со мной на дивание недалеко от камина и после долгого молчания, во время которого я стала чувствовать себя всё более неуютно, он начал говорить.")

CPU times: user 9.85 ms, sys: 482 µs, total: 10.3 ms
Wall time: 301 ms


'TRUE'

In [30]:
%%time 
decision("Он встал и он вышел")

CPU times: user 6.42 ms, sys: 350 µs, total: 6.77 ms
Wall time: 164 ms


'coreference resolution'

In [ ]:
decision("Дима распорядился, чтобы было объявлено, что он покинул страну.")

In [ ]:
decision("Лина хочет, чтобы выиграла")

In [ ]:
decision("Дима сказал маме Кирилла, чтобы она приходила поскорее.")

In [ ]:
decision("Дима сказал маме Кирилла, чтобы приходила поскорее.")

# Classifier

In [ ]:
import numpy as np

In [ ]:
with open(DATA, mode='r', encoding='utf-16') as data:
    df_clas = pd.read_csv(data, sep='\t', header=None, names=['sentence', 'class'])

In [ ]:
df_clas['sentence'] = df_clas['sentence'].apply(lambda x: classifier_preprocessing.preprocess(x))

In [ ]:
df_clas['class'] = df_clas['class'].apply(lambda x: classifier_preprocessing.class2id(x))

In [ ]:
df_clas

In [ ]:
import my_parsers

In [ ]:
%%time
df_clas['conll'] = df_clas['sentence'].apply(lambda x: my_parsers.udpipe_req(x))

In [ ]:
df_clas['cconj'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_conj(x, 'CCONJ'))

In [ ]:
df_clas['sconj'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_conj(x, 'SCONJ'))

In [ ]:
df_clas['tokens'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_tokens(x))

In [ ]:
df_clas['nullp'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_null_pron(x))

In [ ]:
df_clas['overtp'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_overt_pron(x))

In [ ]:
df_clas['adj'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_adjuncts(x))

In [ ]:
df_clas['whq'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_whquestions(x))

In [ ]:
df_clas['relc'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_relclauses(x))

In [ ]:
df_clas['datdep'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_case_nodes(x, 'Dat'))

In [ ]:
df_clas['nomdep'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_case_nodes(x, 'Nom'))

In [ ]:
df_clas['rootlen'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.get_root_length(x))

In [ ]:
df_clas['rootdeps'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_root_dependents(x))

In [ ]:
df_clas['rootpos'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.get_root_pos(x))

In [ ]:
df_clas['rootmatch'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.is_there_exact_string_match(x))

In [ ]:
df_clas['rootdatdep'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_root_case_dep(x, 'Dat'))

In [ ]:
df_clas['rootnomdep'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.count_root_case_dep(x, 'Nom'))

In [ ]:
df_clas['att'] = df_clas['conll'].apply(lambda x: classifier_preprocessing.is_there_att_verb(x))

In [ ]:
df_clas.head()

# Data description

In [ ]:
df_clas = df_clas.drop(['conll', 'sentence'], axis=1)

Number of 'FALSE' sentences

In [ ]:
len(df_clas[df_clas['class'] == 0])

Number of 'TRUE' sentences

In [ ]:
len(df_clas[df_clas['class'] == 1])

Number of 'coreference resolution' sentences

In [ ]:
len(df_clas[df_clas['class'] == 2])

In [ ]:
df_clas = df_clas[df_clas['class'] != 2]

In [ ]:
dataset = df_clas.to_numpy()

In [ ]:
# target, data
target = dataset[:,0]
data = dataset[:,1:]

# Evaluation

In [ ]:
# !pip3 install matplotlib

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density

import matplotlib.pyplot as plt

In [ ]:
def benchmark(clf):
    print('Training: ', clf)
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    precisions = cross_val_score(clf, data, target, cv=cv, scoring='precision')
    recalls = cross_val_score(clf, data, target, cv=cv, scoring='recall')
    f1scores = cross_val_score(clf, data, target, cv=cv, scoring='f1')
    
    precision_mean = precisions.mean()
    precision_std = precisions.std()
    print('Precision - mean: ', precision_mean, ' - std: ', precision_std)
    
    recall_mean = recalls.mean()
    recall_std = recalls.std()
    print('Recall - mean: ', recall_mean, ' - std: ', recall_std)
    
    f1score_mean = f1scores.mean()
    f1score_std = f1scores.std()
    print('F1 score - mean: ', f1score_mean, ' - std: ', f1score_std)
    
    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, precision_mean, recall_mean, f1score_mean

In [ ]:
results = list()
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="sag"), "Ridge Classifier"),
        (Perceptron(max_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(max_iter=50),
         "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False,
                                       tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=500,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=500,
                                       penalty="elasticnet")))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))
results.append(benchmark(ComplementNB(alpha=.1)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,
                                                  tol=1e-3, max_iter=30000))),
  ('classification', LinearSVC(penalty="l2", max_iter=30000))])))

In [ ]:
indices = np.arange(len(results))

results = [[x[i] for x in results] for i in range(4)]
# print(results)

clf_names, precision, recall, f1score = results

plt.figure(figsize=(12, 8))
plt.title("Evaluation")

plt.barh(indices, precision, .2, label="precision", color='navy')
plt.barh(indices + .3, recall, .2, label="recall", color='c')
plt.barh(indices + .6, f1score, .2, label="f1_score", color='darkorange')

plt.yticks(())
plt.legend(loc='best')
plt.subplots_adjust(left=.25)
plt.subplots_adjust(top=.95)
plt.subplots_adjust(bottom=.05)

for i, c in zip(indices, clf_names):
    plt.text(-.3, i, c)

plt.show()